In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets

In [2]:
R = 8.314 #J⋅K−1⋅mol−1
sigma_water = 72* 1e-3 # N m-1
mv = 18 * 1e-3 # Kg mol-1
rho_w = 1000 # Kg m-3

In [3]:
T = widgets.FloatSlider(min=250, max=300, step=0.01, value=288.15, description = 'Temperature') # K
D= widgets.IntRangeSlider(min=1, max = 100, value = [5, 65], description = 'Wet Diameter') # need time 1e-8
d_0 = widgets.FloatSlider(min=40, max=200, step=1, value=41,description='dry diameter') # nm
frac_org = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.5, description='organic fraction') 
delta_min = widgets.FloatSlider(min=0.1, max=0.3, step=0.01, value=0.16, description='delta_min') # nm
'''Inorganics'''
sigma_inorganic = widgets.Dropdown(options=[('AS', 50), ('H2SO4', 49)], description='$\sigma_{inorganic}$') # mN/m
kappa_inorganic = widgets.Dropdown(options= [('AS', 0.58), ('H2SO4', 0.7)], description='$\kappa_{inorganic}$')
rho_inorganic = widgets.Dropdown(options = [('SO4', 1800), ('NO3', 1800), ('Cl',  2200), ('NH4', 1800), ('Na',  2200), ('Ca', 2600), ('OIN', 2600), ('CO3', 2600)], description='$\\rho_{inorganics}$')
'''Organics'''
sigma_organic = widgets.Dropdown(options= [('system2', 30)], description='$\sigma_{organics}$') # mN/m
kappa_organic = widgets.Dropdown(options = [('MSA', 0.53), ('ARO1', 0.1), ('ARO2', 0.1), ('ALK1', 0.1), ('OLE1', 0.1), ('API1', 0.1 ), ('API2', 0.1), ('LIM1', 0.1), ('LIM2', 0.1), ('OC', 0.001)], value = 0.1, description='$\kappa_{organics}$')
rho_organic = widgets.Dropdown(options = [('MSA', 1800), ('ARO1', 1400), ('ARO2', 1400), ('ALK1', 1400), ('OLE1', 1400),('API1', 1400), ('API2', 1400), ('LIM1', 1400), ('LIM2', 1400), ('OC', 1000)], description='$\\rho_{organics}$')

In [12]:
Dw = np.linspace(D.value[0]*1e-8, D.value[1]*1e-8,100)
def S_const_sigma(Dw):
    A_cs = (4 * sigma_water * mv)/(R * T.value * rho_w)
    kappa_cs = (1 - frac_org.value) * kappa_inorganic.value + frac_org.value * kappa_organic.value
    ss_cs = (Dw**3 - ((d_0.value)*1e-9)**3)/(Dw**3 - (1-kappa_cs)*((d_0.value)*1e-9)**3)*np.exp(A_cs/Dw)  
    return ss_cs 

def S_varying_sigma(Dw):
    v_wet = (np.pi * Dw **3)/6
    v_dry = (np.pi * ((d_0.value)*1e-9)**3)/6
    # calculate the minimum shell volume, v_delta
    v_delta = v_wet - (((4 * np.pi) / 3) * (Dw /2 - delta_min.value * 1e-9) ** 3)
    # calculate the total volume of organic, v_beta
    v_beta = frac_org.value * v_dry                                                               
    # calculate the coverage parameter
    c_beta = np.minimum(v_beta / v_delta, 1)                                                   
    f_core_water = ((Dw - 2 * delta_min.value * 1e-9) ** 3 - ((d_0.value) * 1e-9) ** 3)/(Dw - 2 * delta_min.value * 1e-9) ** 3
    f_core_inorganic = 1 - f_core_water
    sigma_core = f_core_water * sigma_water + f_core_inorganic * sigma_inorganic.value * 1e-3
    sigma_shell = sigma_organic.value * 1e-3
    # calculate overall sigma
    sigma = (1 - c_beta) * sigma_core + c_beta * sigma_shell                                    
    A_vs = (4* sigma * mv)/(R * T.value * rho_w)
    kappa_vs = (1 - frac_org.value) * kappa_inorganic.value + frac_org.value * kappa_organic.value
    ss_vs = ((Dw ** 3 - ((d_0.value) * 1e-9) ** 3)/(Dw ** 3 - (1-kappa_vs) * ((d_0.value) * 1e-9) ** 3)) * np.exp(A_vs/Dw)
    om_in = (rho_organic.value * frac_org.value)/(rho_inorganic.value * (1 - frac_org.value))
    return c_beta, sigma, ss_vs, om_in

def plot(T, D, d_0, frac_org, delta_min, sigma_inorganic, kappa_inorganic, sigma_organic, kappa_organic, rho_inorganic, rho_organic):
    # Dw = np.linspace(int(D.value[0])*1e-8, int(D.value[1])*1e-8,100)
    ss_cs = S_const_sigma(Dw)
    c_beta, sigma, ss_vs, om_in = S_varying_sigma(Dw)
    crit_ss = np.max((ss_vs-1)*100)
    max_cs_index = np.argmax(ss_cs)
    max_vs_index = np.argmax(ss_vs)
    high = [Dw[max_cs_index], (ss_cs[max_cs_index]-1)*100]
    low = [Dw[max_vs_index], (ss_vs[max_vs_index]-1)*100]
    delta_D = ((Dw[max_vs_index] - Dw[max_cs_index])/Dw[max_vs_index])*100
    delta_SS = ((ss_vs[max_vs_index] - ss_cs[max_cs_index])/(ss_vs[max_vs_index]-1))*100
    ratio = om_in

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 12))

    ax1.plot(Dw, (ss_cs-1)*100, 'g--', label = 'constant $\sigma$')
    ax1.scatter(Dw[max_cs_index], (ss_cs[max_cs_index]-1)*100, c = 'green', marker='o')
    ax1.plot(Dw, (ss_vs-1)*100, 'b-', label = 'varying $\sigma$')
    ax1.scatter(Dw[max_vs_index], (ss_vs[max_vs_index]-1)*100, c = 'blue', marker='o')
    ax1.text(5e-7, 0, '$\Delta D = %.2f \%%$'%delta_D)
    ax1.text(5e-7, -0.1, '$\Delta SS = %.2f \%%$'%delta_SS)
    ax1.set_ylim(-0.2, 1)
    ax1.set_ylabel("Supersaturation $(\%)$")
    ax1.legend()

    ax2.plot(Dw, sigma*1000, 'b-')
    ax2.axhline(y=72, color= 'green', linestyle='--')
    ax2.set_xlabel('particle diameter')
    ax2.set_ylabel('$\sigma\ (mN\ m^{-1})$')
    ax2.set_xlim(0, 6e-7)
    ax2.set_ylim(20,80)

    ax3 = ax2.twinx()
    ax3.plot(Dw, c_beta)
    ax3.set_ylim(0, 1.1)
    ax3.set_ylabel('surface coverage')
    
    print("Organic/Inorganic mass ratio is "+str(round(ratio,3))+", and critical supersaturation is "+str(round(crit_ss,3))+"%." )


In [13]:
widgets.interact(plot, T=T, D=D, d_0=d_0, frac_org=frac_org, delta_min=delta_min, 
                       sigma_inorganic=sigma_inorganic, kappa_inorganic=kappa_inorganic, rho_inorganic=rho_inorganic, 
                       sigma_organic=sigma_organic, kappa_organic=kappa_organic, rho_organic=rho_organic)

interactive(children=(FloatSlider(value=288.15, description='Temperature', max=300.0, min=250.0, step=0.01), I…

<function __main__.plot(T, D, d_0, frac_org, delta_min, sigma_inorganic, kappa_inorganic, sigma_organic, kappa_organic, rho_inorganic, rho_organic)>